In [1]:
import dspy
from dspy.teleprompt import BootstrapFewShot

local_llm = dspy.LM(
    "openai/Qwen3-VL-8B-Instruct-Q4_K_M.gguf", 
    api_base="http://localhost:8080/v1", 
    api_key="no_key_needed"
)

dspy.configure(lm=local_llm)

In [2]:
# Signatur für die Generierung der Geschichte
class StorySignature(dspy.Signature):
    """Generiert eine kurze, kreative Geschichte basierend auf einer Figur und einem Schauplatz."""
    character = dspy.InputField(desc="Die Hauptfigur der Geschichte.")
    setting = dspy.InputField(desc="Der Schauplatz, an dem die Geschichte spielt.")
    story = dspy.OutputField(desc="Eine kurze, kreative Geschichte mit etwa 50 Wörtern.")

# Signatur für die Zusammenfassung der Geschichte
class SummarizationSignature(dspy.Signature):
    """Fasst eine gegebene Geschichte in einem einzigen Satz zusammen."""
    story = dspy.InputField(desc="Die zu zusammenfassende Geschichte.")
    summary = dspy.OutputField(desc="Eine Zusammenfassung in einem Satz.")

In [3]:
class StorySummarizer(dspy.Module):
    def __init__(self):
        super().__init__()
        # Deklaration der beiden Module im Konstruktor
        self.story_generator = dspy.Predict(StorySignature)
        self.summarizer = dspy.Predict(SummarizationSignature)

    def forward(self, character, setting):
        # Schritt 1: Geschichte generieren
        # Das erste Modul wird mit den ursprünglichen Eingaben aufgerufen
        generated_story = self.story_generator(character=character, setting=setting)

        # Schritt 2: Geschichte zusammenfassen
        # Die Ausgabe von Schritt 1 ('story') wird als Eingabe für Schritt 2 verwendet
        final_summary = self.summarizer(story=generated_story.story)

        return final_summary

In [4]:

# Instanziieren und Ausführen des Programms
summarizer_program = StorySummarizer()
result = summarizer_program(character="ein neugieriger Roboter", setting="auf dem Mars")

In [5]:
# Ausgabe des Ergebnisses
print(f"Charakter: ein neugieriger Roboter")
print(f"Schauplatz: auf dem Mars")
print("-" * 20)
print(f"Zusammenfassung: {result.summary}")

Charakter: ein neugieriger Roboter
Schauplatz: auf dem Mars
--------------------
Zusammenfassung: Ein neugieriger Roboter entdeckt auf dem Mars mysteriöse, verschlüsselte Signale, die darauf hindeuten, dass er nicht allein ist und der Planet mehr Geheimnisse birgt, als er dachte.


In [7]:
local_llm.inspect_history(n=2)





[2025-11-11T15:54:17.146712]

System message:

Your input fields are:
1. `character` (str): Die Hauptfigur der Geschichte.
2. `setting` (str): Der Schauplatz, an dem die Geschichte spielt.
Your output fields are:
1. `story` (str): Eine kurze, kreative Geschichte mit etwa 50 Wörtern.
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## character ## ]]
{character}

[[ ## setting ## ]]
{setting}

[[ ## story ## ]]
{story}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Generiert eine kurze, kreative Geschichte basierend auf einer Figur und einem Schauplatz.


User message:

[[ ## character ## ]]
ein neugieriger Roboter

[[ ## setting ## ]]
auf dem Mars

Respond with the corresponding output fields, starting with the field `[[ ## story ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## story ## ]]
Ein neugieriger Roboter erkundete den Mars, wo er seltsame Steinforma